# Rotman Data Science Competition
### Section 6: Picking Top 1000 Products

## 0. Imports


In [179]:
import pandas as pd

## 1. Load Data


In [180]:
# Make dummy substitute data file
def make_dummy_substitute_data() -> bool:
    DATA_PATH = "data/mma_mart.csv"
    DUMMY_PATH = "data/data_with_substitutes_dummy.csv"
    data = pd.read_csv(DATA_PATH)
    data["substitute"] = data["product_id"]
    data["frozen"] = [False] * data.shape[0]
    data["refrigerated"] = [False] * data.shape[0]
    data.to_csv(DUMMY_PATH, index=False)
    return True
# assert make_dummy_substitute_data()

In [181]:
DATA_WTH_SUBS_PATH = "data/substitute_df.csv"
data = pd.read_csv(DATA_WTH_SUBS_PATH)
data.head()

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated
0,Banana,14494,10,24852,24,fresh fruits,4,produce,24852,False,False
1,Bag of Organic Bananas,11694,1,13176,24,fresh fruits,4,produce,13176,False,False
2,Organic Strawberries,8081,10,21137,24,fresh fruits,4,produce,21137,False,False
3,Organic Baby Spinach,7369,3,21903,123,packaged vegetables fruits,4,produce,21903,False,False
4,Organic Hass Avocado,6411,1,47209,24,fresh fruits,4,produce,47209,False,False


## 2. Baseline Model

In [182]:
def baseline(data: pd.DataFrame, k: int, use_substitutes=True) -> pd.DataFrame:
    """ Return the top k items by sales

    Precondition:
    k is less than the number of items in the dataset
    """
    # Count number of orders for each product accounting for substitutes
    if use_substitutes:
        # Count number of unique orders for each product (accounting for substitutes)
        sales = pd.DataFrame(data.groupby("substitute")["order_id"].nunique())
    else:
        # Count number of orders for each product. Uniqueness is given.
        sales = pd.DataFrame(data.groupby("product_id")["order_id"].count())

    # Rename column to sales and sort by sales
    sales.rename(columns = {"order_id": "sales"}, inplace=True)
    sales.sort_values(by="sales", ascending=False, inplace=True)
    sales.reset_index(inplace=True)

    # Catch errors for debugging purposes
    if k > sales.shape[0]:
        raise ValueError("k must be less than the number of items in the dataset")

    # Return top k items by sales (i.e. top k items with most orders)
    return sales.iloc[:k]

In [183]:
# Test baseline model
k = 1000
baseline_aisle = baseline(data, k, use_substitutes=True)["substitute"].tolist()
baseline(data, k, use_substitutes=True).head()

,substitute,sales
0,49075,116
1,40709,115
2,21417,115
3,31433,114
4,605,113


## 3. Processing Data For Metric B Optimization


In [184]:
def add_met_b_column(data, use_substitutes=True) -> None:
    """ Add a column to the data that is the metric b impact score for each product calculated over the full dataset """
    order_id_to_order_size = data.groupby("order_id")["order_id"].count()
    mma_data_aug = data.copy()
    mma_data_aug["order_size"] = mma_data_aug["order_id"].map(order_id_to_order_size)

    mma_data_aug.loc[:, "portion_of_order"] = 1 / mma_data_aug["order_size"]

    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    id_to_met_b_score = mma_data_aug.groupby(column)["portion_of_order"].sum()
    data["met_b_score"] = data[column].map(id_to_met_b_score)
    return None

In [185]:
add_met_b_column(data)

In [186]:
data.sort_values(by="substitute", ascending=False).head(100)

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated,met_b_score
18436,Cucumber,4,3735,6581,83,fresh vegetables,4,produce,49683,False,False,1.625000
8548,Cucumber Seedless,15,6037,34809,83,fresh vegetables,4,produce,49683,False,False,1.625000
15,Cucumber Kirby,2905,1,49683,83,fresh vegetables,4,produce,49683,False,False,1.625000
17699,Chicken and Cheese Taquitos,4,39357,11716,129,frozen appetizers sides,1,frozen,49678,True,False,2.325000
32365,Pierogi & Onion,1,61336,46551,38,frozen meals,1,frozen,49678,True,False,2.325000
...,...,...,...,...,...,...,...,...,...,...,...,...
13743,Mandarin Orange Sparkling Water,7,6460,5711,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
12717,Tangerine Lemongrass Sparkling Water,8,14608,39199,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
16009,Mandarin Orange Sparkling Seltzer Water,5,15567,11194,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
24391,Black Raspberry Water,2,89519,33142,100,missing,21,missing,49520,False,False,15.369553


## 4. Optimization and Finding Top 1000 Products


In [187]:
# trim data
my_data = data.drop(columns=["aisle_id", "aisle", "department_id", "department"])
my_data.head()

,product_name,total_quantity_sold,order_id,product_id,substitute,frozen,refrigerated,met_b_score
0,Banana,14494,10,24852,24852,False,False,22.188600
1,Bag of Organic Bananas,11694,1,13176,13176,False,False,3.503968
2,Organic Strawberries,8081,10,21137,21137,False,False,17.255952
3,Organic Baby Spinach,7369,3,21903,21903,False,False,10.514652
4,Organic Hass Avocado,6411,1,47209,47209,False,False,4.249242


In [188]:
def get_max_aisle(data: pd.DataFrame, use_substitutes=True) -> list:
    """ Return the optimal aisle of size 1000.
    """
    return _max_aisle_recur_helper(data, [], 1000, use_substitutes=use_substitutes)

In [189]:
def get_top_product_by_sales(data: pd.DataFrame, use_substitutes=True) -> list:
    """ Return the top item by sales in data. """
    # Count number of orders for each product
    if use_substitutes:
        # Count number of unique orders for each product (accounting for substitutes)
        sales = pd.DataFrame(data.groupby("substitute")["order_id"].nunique())
    else:
        # Count number of orders for each product. Uniqueness is given.
        sales = pd.DataFrame(data.groupby("product_id")["order_id"].count())

    # Rename column to sales
    sales.rename(columns = {"order_id": "sales"}, inplace=True)

    # Get top item by sales
    sales.reset_index(inplace=True) # Reset index so we don't get multiple items if there is a tie

    # Get top item
    if use_substitutes:
        top_item =  sales.loc[sales["sales"].idxmax()]["substitute"]
    else:
        top_item =  sales.loc[sales["sales"].idxmax()]["product_id"]

    return top_item

In [190]:
def _max_aisle_recur_helper(data: pd.DataFrame, aisle: list, k: int, use_substitutes=True) -> list:
    """ Return the aisle of size k by the process of picking the item with the highest metric b score, removing all orders that contain that item, and repeating until the aisle is of size k.

    Precondition:
    - aisle must be a subset of the test data
    """
    # Base case
    if len(aisle) == k:
        return aisle

    else:
        # Get top items by metric b score
        top_items = data[data["met_b_score"] == data["met_b_score"].max()]

        # If there is a tie, break it by picking the item with the highest sales
        top_item = get_top_product_by_sales(top_items, use_substitutes=use_substitutes)

        # Add top item to aisle
        aisle.append(top_item)

        if use_substitutes:
            column = "substitute"
        else:
            column = "product_id"

        # Filter data to only include orders that do not contain the top item
        top_item_orders = data[data[column] == top_item]
        data_excluding_top_item = data[~data["order_id"].isin(top_item_orders["order_id"])]

        # Recursively call helper function
        return _max_aisle_recur_helper(data_excluding_top_item, aisle, k)

In [191]:
# Get the Aisle
my_max_aisle = get_max_aisle(my_data, use_substitutes=True)
my_aisle_df = data[data["substitute"].isin(my_max_aisle)]

## 5. Combining Code

In [192]:
def make_1000_items(data: pd.DataFrame, use_substitutes=True) -> list:
    """ Make our aisle of 1000 items """
    data_copy = data.copy()
    add_met_b_column(data_copy)
    return get_max_aisle(data_copy, use_substitutes=True)

## 6. Evaluation


In [193]:
def metric_a(test_data: pd.DataFrame, insta_aisle: list, use_substitutes=True) -> float:
    """ Return the number of orders that utilize the in-aisle items as a percentage of the total number of orders in the test_data dataset.

    Precondition:
    len(insta_aisle) == 1000
    len(test_data) > 0
    """
    # Account for whether we are using substitutes or not
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    # Count the orders that utilize in-aisle items
    ## Filter the data to only include orders that purchased at least one item in the aisle
    purchases_in_aisle = test_data[test_data[column].isin(insta_aisle)]

    ## Count the number of orders that include at least one item in the aisle
    orders_in_aisle = purchases_in_aisle["order_id"].nunique()

    ## Count the total number of orders in the test data
    total_orders = test_data["order_id"].nunique()

    # Return the percentage of orders that utilize in-aisle items
    result = (orders_in_aisle / total_orders) * 100


    # Check for errors for debugging purposes
    if total_orders <= 0:
        raise ValueError("There must be at least one order in the test data")

    if len(insta_aisle) != 1000:
        raise ValueError("insta_aisle must contain 1000 product ids")


    return result


In [194]:
def metric_b(test_data: pd.DataFrame, insta_aisle: list, use_substitutes=True) -> float:
    """ Calculate the percentage of items in each order that utilize in-aisle items and return the average of these percentages.

    Precondition:
    len(insta_aisle) == 1000
    """
    # Check for errors for debugging purposes
    if len(insta_aisle) != 1000:
        raise ValueError("There should be 1000 items in the aisle")

    # Account for whether we are using substitutes or not
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    # Copy data to prevent altering test_data
    test_data_copy = test_data.copy()

    # Add column that is 1 if the product id is in insta_aisle and 0 if not
    test_data_copy["is_utilized"] = test_data_copy[column].isin(insta_aisle).astype(int)

    # Sum the new column and divide by total number of items in order
    ## Get a map mapping order_id to the number of items in the order that are utilized
    order_to_ultilized_items = test_data_copy.groupby("order_id")["is_utilized"].sum()

    ## Get a map mapping order_id to the total number of items in the order
    order_to_total_items = test_data_copy.groupby("order_id")["order_id"].count()

    ## Divide the two maps to get order to the percentage of items in the order that are utilized
    order_to_precent_ultilization = order_to_ultilized_items / order_to_total_items


    # Return average of percentage ultilization over all orders
    return order_to_precent_ultilization.mean() * 100


In [195]:
def metric_average(test_data: pd.DataFrame, insta_aisle: list) -> float:
    """ Return the average of metric a and metric b

    Precondition:
    len(insta_aisle) == 1000
    """
    return (metric_a(test_data, insta_aisle) + metric_b(test_data, insta_aisle)) / 2

In [196]:
metric_average(data, my_max_aisle)

73.99193621720606

In [197]:
metric_average(data, baseline_aisle)

73.05470432918452

In [198]:
def get_num_frozen_and_refrigerated(data, aisle, use_substitutes=True) -> tuple:
    """ Return the number of frozen and refrigerated items in the aisle """
    # Get the number of frozen and refrigerated items in the aisle
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"
    frozen = data[data[column].isin(aisle)]["frozen"].astype(int).sum()
    refrigerated = data[data[column].isin(aisle)]["refrigerated"].astype(int).sum()
    return frozen, refrigerated

In [199]:
get_num_frozen_and_refrigerated(data, my_max_aisle)

(2076, 3106)

In [200]:
my_aisle_df

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated,met_b_score
0,Banana,14494,10,24852,24,fresh fruits,4,produce,24852,False,False,22.188600
2,Organic Strawberries,8081,10,21137,24,fresh fruits,4,produce,21137,False,False,17.255952
3,Organic Baby Spinach,7369,3,21903,123,packaged vegetables fruits,4,produce,21903,False,False,10.514652
6,Large Lemon,4688,28,47626,24,fresh fruits,4,produce,47626,False,False,6.081728
7,Strawberries,4340,47,16797,24,fresh fruits,4,produce,16797,False,False,13.609957
...,...,...,...,...,...,...,...,...,...,...,...,...
35063,Buttercup Squash,1,28619,33235,83,fresh vegetables,4,produce,45007,False,False,13.917106
35064,"Makeup Dissolving Facial Cleansing Wipes, Oil-...",1,99845,13839,73,facial care,11,personal care,10732,False,False,5.100000
35065,Seasoned Lemon & Pepper Sensations Tuna Medley,1,97599,44960,95,canned meat seafood,15,canned goods,19691,False,False,29.092460
35068,Major Grey's Mango Chutney,1,64347,5480,88,spreads,13,pantry,15906,False,False,13.166667


In [201]:
data[data["substitute"].isin(baseline_aisle)]

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated,met_b_score
0,Banana,14494,10,24852,24,fresh fruits,4,produce,24852,False,False,22.188600
2,Organic Strawberries,8081,10,21137,24,fresh fruits,4,produce,21137,False,False,17.255952
3,Organic Baby Spinach,7369,3,21903,123,packaged vegetables fruits,4,produce,21903,False,False,10.514652
6,Large Lemon,4688,28,47626,24,fresh fruits,4,produce,47626,False,False,6.081728
7,Strawberries,4340,47,16797,24,fresh fruits,4,produce,16797,False,False,13.609957
...,...,...,...,...,...,...,...,...,...,...,...,...
35063,Buttercup Squash,1,28619,33235,83,fresh vegetables,4,produce,45007,False,False,13.917106
35064,"Makeup Dissolving Facial Cleansing Wipes, Oil-...",1,99845,13839,73,facial care,11,personal care,10732,False,False,5.100000
35065,Seasoned Lemon & Pepper Sensations Tuna Medley,1,97599,44960,95,canned meat seafood,15,canned goods,19691,False,False,29.092460
35068,Major Grey's Mango Chutney,1,64347,5480,88,spreads,13,pantry,15906,False,False,13.166667


In [202]:
metric_a(data, my_max_aisle)

78.6016617024612

In [203]:
metric_b(data, my_max_aisle)

69.38221073195092

In [204]:
metric_a(data, baseline_aisle)

77.48863458222291

In [205]:
metric_b(data, baseline_aisle)

68.62077407614613

## 7. Getting Theoretical Maximum Aisles


In [206]:
def max_metric_a_aisle(data: pd.DataFrame) -> list:
    """ Return the aisle that maximizes metric a over the data. Do this by first getting the item with most sales, then getting the item with most sales in orders that do not contain the first item, and so on. For why this produces max list, see proof writeup.
    """
    return _max_metric_a_recur_helper(data, [])

def _max_metric_a_recur_helper(data: pd.DataFrame, aisle: list) -> list:
    """ Return the aisle that maximizes metric a over the data. Do this by first getting the item with most sales, then getting the item with most sales in orders that do not contain the first item, and so on.

    Precondition:
    - data must be a subset of the test data
    - aisle must be a subset of the top 1000 items by sales
    """
    # Base case
    if len(aisle) == 1000:
        return aisle

    else:
        # Get the top item by sales in data
        top_item = get_top_product(data)

        # Add top item to aisle
        aisle.append(top_item)

        # Filter data to only include orders that do not contain the top item
        top_item_orders = data[data["substitute"] == top_item]
        data_excluding_top_item = data[~data["order_id"].isin(top_item_orders["order_id"])]

        # Recursively call helper function
        return _max_metric_a_recur_helper(data_excluding_top_item, aisle)

def get_top_product(data: pd.DataFrame) -> list:
    """ Return the top item by sales in data. """
    # Count number of orders for each product
    sales = pd.DataFrame(data.groupby("substitute")["order_id"].nunique())

    # Rename column to sales
    sales.rename(columns = {"order_id": "sales"}, inplace=True)

    # Get top item by sales
    sales.reset_index(inplace=True) # Reset index so we don't get multiple items if there is a tie
    top_item =  sales.loc[sales["sales"].idxmax()]["substitute"]

    return top_item

In [207]:
%%time
# Max Metric a for the test set
met_a_top_1000 = max_metric_a_aisle(data)
print(f"Max Metric a: {metric_a(data, met_a_top_1000)}")

Max Metric a: 79.01970005748026
CPU times: user 5.96 s, sys: 1.94 s, total: 7.9 s
Wall time: 7.97 s


In [208]:
def max_metric_b_aisle(data: pd.DataFrame) -> list:
    """ Return the aisle that maximizes metric b over the data. Do this by picking the thousand items with the highest metric b score.
    """
    data_copy = data.copy()
    add_met_b_column(data_copy)

    met_b_top_1000 = data_copy.sort_values(by="met_b_score", ascending=False)
    met_b_top_1000.drop(columns="order_id", inplace=True)
    met_b_top_1000.drop_duplicates(subset="substitute", inplace=True)
    met_b_top_1000.reset_index(drop=True, inplace=True)
    met_b_top_1000 = met_b_top_1000.iloc[:1000]["substitute"].tolist()
    return met_b_top_1000

In [209]:
%%time
# Max Metric b for the test set
met_b_top_1000 = max_metric_b_aisle(data)
print(f"Max Metric b: {metric_b(data, met_b_top_1000)}")

Max Metric b: 69.38221073195092
CPU times: user 35.4 ms, sys: 2.27 ms, total: 37.7 ms
Wall time: 36.4 ms


In [210]:
max_average = (metric_a(data, met_a_top_1000) + metric_b(data, met_b_top_1000))/2
max_average

74.20095539471559

In [212]:
data.sort_values(by=["substitute", 'met_b_score'], ascending=False).head(1000)

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated,met_b_score
15,Cucumber Kirby,2905,1,49683,83,fresh vegetables,4,produce,49683,False,False,1.625000
8548,Cucumber Seedless,15,6037,34809,83,fresh vegetables,4,produce,49683,False,False,1.625000
18436,Cucumber,4,3735,6581,83,fresh vegetables,4,produce,49683,False,False,1.625000
11553,Large Chicken & Cheese Taquitos,9,3785,49678,129,frozen appetizers sides,1,frozen,49678,True,False,2.325000
13780,Beef & Cheese Taquitos,7,3272,17936,38,frozen meals,1,frozen,49678,True,False,2.325000
...,...,...,...,...,...,...,...,...,...,...,...,...
33705,"Iso Maki, Seaweed Wrapped Rice Cracker",1,22327,48992,66,asian foods,6,international,48595,False,False,51.111303
34240,Ritz Crackerfuls Classic Cheddar Filled Crackers,1,75998,12593,78,crackers,19,snacks,48595,False,False,51.111303
34275,Rice Crackers Cheese,1,40989,20349,78,crackers,19,snacks,48595,False,False,51.111303
34916,Savory Cracker Mix,1,30280,36858,100,missing,21,missing,48595,False,False,51.111303
